In [6]:
import math
import os

import matplotlib.pyplot as plt
import numpy as np
import preparers

In [7]:
df_snippet, df_eyetracking = preparers.get_data(16)

(1/6) Construct Paths
(2/6) Read Eye Tracker Data
(3/6) Transform Eye Tracker Data
(4/6) Normalize Eye Tracker Time
(5/6) Read PsychoPy Data
(6/6) Transform PsychoPy Data


In [8]:
def create_circular_mask(h, w, center=None, radius=None):

    if center is None:  # use the middle of the image
        center = (int(w / 2), int(h / 2))
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    x_low = max(center[0] - radius, 0.0)
    x_high = min(center[0] + radius, w)
    y_low = max(center[1] - radius, 0.0)
    y_high = min(center[1] + radius, w)
    X = X[:, x_low:x_high]
    Y = Y[y_low:y_high]
    dist_from_center = np.full([h, w], radius + 1)
    dist_from_center[y_low:y_high, x_low:x_high] = np.sqrt((X - center[0]) ** 2 + (Y - center[1]) ** 2)

    mask = dist_from_center <= radius
    return mask


def create_heapmap(eye_x, eye_y, eye_valid, d_time, h, w, radius=30):
    hmap = np.full([h, w], 0.0)
    heat = np.full([h, w], d_time)
    for idx in range(len(eye_x)):
        if eye_valid[idx] == 1:
            x_prev = eye_x[idx]
            y_prev = eye_y[idx]
            x = max(0, min(w, eye_x[idx]))
            y = max(0, min(h, eye_y[idx]))
            if x_prev != x or y_prev != y:
                continue
            mask = create_circular_mask(h, w, (x, y), radius)
            hmap[mask] += heat[mask]
    return hmap


def create_sequence_diagram_y(eye_y, eye_valid, h, w, offset=0, step=0.5):
    sequence = np.full([h, w], 0.0)
    current_x = offset
    current = None
    x_step = step

    for idx, y in enumerate(eye_y):
        if eye_valid[idx] == 1:
            current = (min(max(0, y), h - 1), min(max(0, current_x), w - 1))
            break

    for idx, y in enumerate(eye_y):
        if eye_valid[idx] == 1:
            old_x = current[1]
            old_y = current[0]
            next_y = min(max(0, y), h - 1)
            next_x = min(max(0, current[1] + x_step), w - 1)
            sequence[old_y, int(old_x) : int(next_x)] = 1.0
            sequence[old_y:next_y, int(next_x)] = 1.0
            sequence[next_y:old_y, int(next_x)] = 1.0
            current = (next_y, next_x)

    return sequence


def create_sequence_diagram_x(eye_x, eye_valid, h, w, offset=0, step=0.5):
    sequence = np.full([h, w], 0.0)
    current_y = offset
    current = None
    y_step = step

    for idx, x in enumerate(eye_x):
        if eye_valid[idx] == 1:
            current = (min(max(0, current_y), h - 1), min(max(0, x), w - 1))
            break

    for idx, x in enumerate(eye_x):
        if eye_valid[idx] == 1:
            old_x = current[1]
            old_y = current[0]
            next_y = min(max(0, current[0] + y_step), h - 1)
            next_x = min(max(0, x), w - 1)

            sequence[int(old_y) : int(next_y), old_x] = 1.0
            sequence[int(next_y), old_x:next_x] = 1.0
            sequence[int(next_y), next_x:old_x] = 1.0
            current = (next_y, next_x)

    return sequence

In [5]:
dt = 0.008595
display_w = 1920
display_h = 1080

prefix = "./filteredData/16/"
os.makedirs(prefix, exist_ok=True)
for idx, row in df_snippet.iterrows():
    image = "./images/" + row["Snippet"] + ".png"
    image_name = row["Snippet"]
    print("Calculate " + image_name)

    image = plt.imread(image)
    start = row["Start"]
    stop = row["Stop"]
    df_eye = df_eyetracking[(df_eyetracking["time"] >= start) & (df_eyetracking["time"] < stop)]

    left_color = (1.0, 0.0, 0.0)
    right_color = (0.0, 1.0, 0.0)

    #########################################################################
    # transform data to screen coordinates
    print("\t(1/5): Transforming Eytracking Data to Screen Coordinates")
    df_eye["l_display_x"] = df_eye["l_display_x"].apply(lambda x: x * display_w if (math.isnan(x) == False) else -1)
    df_eye["l_display_x"] = df_eye["l_display_x"].astype(int)

    df_eye["r_display_x"] = df_eye["r_display_x"].apply(lambda x: x * display_w if (math.isnan(x) == False) else -1)
    df_eye["r_display_x"] = df_eye["r_display_x"].astype(int)

    df_eye["l_display_y"] = df_eye["l_display_y"].apply(lambda x: x * display_h if (math.isnan(x) == False) else -1)
    df_eye["l_display_y"] = df_eye["l_display_y"].astype(int)

    df_eye["r_display_y"] = df_eye["r_display_y"].apply(lambda x: x * display_h if (math.isnan(x) == False) else -1)
    df_eye["r_display_y"] = df_eye["r_display_y"].astype(int)

    # reset the index for further processing
    df_eye = df_eye.reset_index()

    #########################################################################
    # generate heat maps
    print("\t(2/5): Generate Heat Maps")
    dre = create_heapmap(df_eye["r_display_x"], df_eye["r_display_y"], df_eye["r_valid"], dt, display_h, display_w)
    dle = create_heapmap(df_eye["l_display_x"], df_eye["l_display_y"], df_eye["l_valid"], dt, display_h, display_w)

    # normalize heat
    dre = dre / dre.max()
    dle = dle / dle.max()

    dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
    dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

    data_right_heat = np.moveaxis(dre, 0, 2)
    data_left_heat = np.moveaxis(dle, 0, 2)

    #########################################################################
    # create sequence diagrams with vertical movement
    print("\t(3/5): Generate Sequence Diagrams for Y Movement")
    dre = create_sequence_diagram_y(df_eye["r_display_y"], df_eye["r_valid"], display_h, display_w)
    dle = create_sequence_diagram_y(df_eye["l_display_y"], df_eye["l_valid"], display_h, display_w)

    dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
    dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

    data_right_seq_y = np.moveaxis(dre, 0, 2)
    data_left_seq_y = np.moveaxis(dle, 0, 2)

    #########################################################################
    # create sequence diagrams with vertical movement
    print("\t(4/5): Generate Sequence Diagrams for X Movement")
    dre = create_sequence_diagram_x(df_eye["r_display_x"], df_eye["r_valid"], display_h, display_w)
    dle = create_sequence_diagram_x(df_eye["l_display_x"], df_eye["l_valid"], display_h, display_w)

    dre = np.array([dre * right_color[0], dre * right_color[1], dre * right_color[2], dre])
    dle = np.array([dle * left_color[0], dle * left_color[1], dle * left_color[2], dle])

    data_right_seq_x = np.moveaxis(dre, 0, 2)
    data_left_seq_x = np.moveaxis(dle, 0, 2)

    #########################################################################
    # plot data
    print("\t(5/5): Generate Plot")
    fig, axs = plt.subplots(2, 5)

    # heatmaps
    ax = axs[0, 0]
    ax.set_axis_off()
    ax.imshow(data_right_heat, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[0, 1]
    ax.set_axis_off()
    ax.imshow(data_left_heat, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[1, 0]
    ax.set_axis_off()
    ax.imshow(data_left_heat, zorder=2, alpha=0.5)
    ax.imshow(data_right_heat, zorder=3, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[1, 1]
    ax.set_axis_off()

    # sequence y
    ax = axs[0, 2]
    ax.set_axis_off()
    ax.imshow(data_right_seq_y, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[0, 3]
    ax.set_axis_off()
    ax.imshow(data_left_seq_y, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[0, 4]
    ax.set_axis_off()
    ax.imshow(data_left_seq_y, zorder=2, alpha=0.5)
    ax.imshow(data_right_seq_y, zorder=3, alpha=0.5)
    ax.imshow(image, zorder=1)

    # sequence x
    ax = axs[1, 2]
    ax.set_axis_off()
    ax.imshow(data_right_seq_x, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[1, 3]
    ax.set_axis_off()
    ax.imshow(data_left_seq_x, zorder=2, alpha=0.5)
    ax.imshow(image, zorder=1)

    ax = axs[1, 4]
    ax.set_axis_off()
    ax.imshow(data_left_seq_x, zorder=2, alpha=0.5)
    ax.imshow(data_right_seq_x, zorder=3, alpha=0.5)
    ax.imshow(image, zorder=1)

    # save plot

    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(prefix + image_name + ".png", dpi=3000)
    plt.close()

Calculate Eyetracking
	(1/5): Transforming Eytracking Data to Screen Coordinates
	(2/5): Generate Heat Maps
	(3/5): Generate Sequence Diagrams for Y Movement
	(4/5): Generate Sequence Diagrams for X Movement
	(5/5): Generate Plot
Calculate Satz1
	(1/5): Transforming Eytracking Data to Screen Coordinates
	(2/5): Generate Heat Maps
	(3/5): Generate Sequence Diagrams for Y Movement
	(4/5): Generate Sequence Diagrams for X Movement
	(5/5): Generate Plot
Calculate Satz2
	(1/5): Transforming Eytracking Data to Screen Coordinates
	(2/5): Generate Heat Maps
	(3/5): Generate Sequence Diagrams for Y Movement
	(4/5): Generate Sequence Diagrams for X Movement
	(5/5): Generate Plot
Calculate Satz3
	(1/5): Transforming Eytracking Data to Screen Coordinates
	(2/5): Generate Heat Maps
	(3/5): Generate Sequence Diagrams for Y Movement
	(4/5): Generate Sequence Diagrams for X Movement
	(5/5): Generate Plot
Calculate Satz4
	(1/5): Transforming Eytracking Data to Screen Coordinates
	(2/5): Generate Heat M